[Solution of the Poisson equation within the domain \[0,1\]](https://github.com/nithinadidela/poisson-eq-FDM/blob/master/Thomas-TDMA.py)

## EXACT SOLUTION

In [ ]:
import numpy as np
import math as m
import scipy.misc as sm
from numba import jit, f8

@jit(f8(f8, f8, f8, f8))
def P0(z, Bi, δ, Gr):
    p1 = -2*Bi*(δ**2)*(z**4)/((1+Bi)*m.factorial(4))
    p2 = 2*(δ**2)*(z**3)/((1+Bi)*m.factorial(3))
    p3 = ((δ**2)*Bi*(Bi+3)-3*Bi*(Bi+1))*(z**2)/(12*((Bi+1)**2))
    p4 = ((3*(Bi+1)-(δ**2)*(Bi+3))/(6*((Bi+1)**2)) - (δ**2)*(4*Bi+15)/(60*(1+Bi)))*z
    return p1 + p2 + p3 + p4 + 1

@jit(f8(f8, f8, f8, f8))
def P11(z, Bi, δ, Gr):
    p1 = Bi*(z**2)/((1+Bi)*m.factorial(2))
    p2 = -z/(1+Bi)
    return p1+p2

@jit(f8(f8, f8, f8, f8))
def T0(z, Bi, δ, Gr):
    t1 = -(Bi*(δ**2)*(z**3))/((1+Bi)*m.factorial(3))
    t2 = ((δ**2)*(z**2))/((1+Bi)*m.factorial(2))
    t3 = ((δ**2)*Bi*(Bi+3)-3*Bi*(Bi+1))*z/(6*((Bi+1)**2))
    t4 = (3*(Bi+1)-(δ**2)*(Bi+3))/(6*((Bi+1)**2)) 
    return t1 + t2 + t3 + t4

def dP0(z, Bi, δ, Gr):
    return sm.derivative(lambda z: P0(z, Bi, δ, Gr), z, dx=0.001)

In [ ]:
import matplotlib.pyplot as plt
from numba import jit, f8

class Parameters:
    def __init__(self, Bi, δ, Gr):
        self.Bi = Bi
        self.δ = δ
        self.Gr = Gr

def PrintExactSolution(exact_solution, parameters):
    l=1.0			     # length of the domain
    n=100			     # number of mesh divisions (for first question)	
    del_z=l/(n-1)		 # mesh size

    z_values = np.zeros(n)
    e_values = np.zeros(n)

    z_values[0] = -1
    e_values[0] = exact_solution(z_values[0], parameters.Bi, parameters.δ, parameters.Gr)

    for i in range(1, len(z_values)):
        z_values[i]=z_values[i-1]+del_z
        e_values[i]=exact_solution(z_values[i], parameters.Bi, parameters.δ, parameters.Gr)

    plt.plot(z_values, e_values, color='blue', markerfacecolor='blue', linestyle='dashed')
    plt.title('Exact solution (Gr={})'.format(parameters.Gr))
    plt.xlabel('Domain')
    plt.ylabel('Exact Solution')
    plt.show()

In [ ]:
PrintExactSolution(P0, Parameters(Bi=2, δ=1.5, Gr=0))

In [ ]:
PrintExactSolution(P11, Parameters(Bi=-3, δ=0, Gr=0))

In [ ]:
PrintExactSolution(T0, Parameters(Bi=-3, δ=0.01, Gr=0))

In [ ]:
PrintExactSolution(dP0, Parameters(Bi=-3, δ=0.01, Gr=0))

## NUMERICAL SOLUTION

In [ ]:
import numba as nb
import math as mt

@nb.jit(f8(nb.f8, nb.f8, nb.f8, nb.f8, nb.f8))
def CalcInitialConditions(z, Bi, δ, Gr, ω):
    C1 = (dP0(z, Bi, δ, Gr) - T0(z, Bi, δ, Gr)) / (δ**2)

    a = 1
    b = -Gr*ω
    c = -1
    D = ((b**2)-4*a*c)/2*a

    k1 = (-b + mt.sqrt(D))/2*a
    k2 = (-b - mt.sqrt(D))/2*a

    λ1 = mt.exp(-k2)/(mt.exp(-k2) - mt.exp(-k1))
    λ2 = -mt.exp(-k1)/(mt.exp(-k2) - mt.exp(-k1))

    return λ1*mt.exp(k1*z) - λ2*mt.exp(k2*z)

In [ ]:
@nb.jit(nb.types.Tuple((nb.f8[:], nb.f8[:]))(nb.f8, nb.f8, nb.f8))
def CalcFirstEquation(δ, Bi, Gr):   
    l=1.0
    n=100
    h=l/(n-1)

    τ = 1e-4
    ε = 1e-5

    z=np.zeros(n)
    α = np.zeros(n+1)
    β = np.zeros(n+1)
    ω = np.zeros(n)

    z[0] = -1
    ω[0] = 0 #CalcInitialConditions(z[0], Bi, δ, Gr, 0.5)

    α[0] = 0
    β[0] = 0

    for i in range(1, n):
        z[i]=i*h + z[0]

        ω_t1 = ω[i-1]
        ω_t2 = 0

        while True:
            C1 = dP0(z[i], Bi, δ, Gr) - T0(z[i], Bi, δ, Gr)

            Ai = ((δ**2)/(h**2) + (Gr*(δ**2)*ω_t1/(2*h)))
            Bi = -2*(δ**2)/(h**2) - 1/τ
            Ci = ((δ**2)/(h**2) - (Gr*(δ**2)*ω_t1/(2*h)))
            Fi =  C1 - ω_t1/τ

            α[i+1] = -Ci/(α[i]*Ai + Bi)
            β[i+1] = (Fi-Ai*β[i])/(α[i]*Ai + Bi)
            ω_t2 = α[i+1]*ω_t1 + β[i+1]

            if abs(ω_t1 - ω_t2) <= ε:
                break

            ω_t1 = ω_t2

        ω[i] = ω_t2
    return z, ω

In [ ]:
δ=1
Bi=-3
Gr=0.03

z, ω = CalcFirstEquation(δ, Bi, Gr)

plt.plot(z, ω, color='blue', markerfacecolor='blue', linestyle='dashed')
plt.title('Numerical solution (Gr={})'.format(Gr))
plt.xlabel('Domain')
plt.ylabel('Numerical Solution')
plt.show()

In [ ]:
CalcInitialConditions(δ=0.01, Bi=2, Gr=0.03, z=-1, ω=0)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def FindSolution(z, v_initial, A_func, B_func, C_func, F_func, β0):   
    ε = 1e-5

    α = np.zeros(n+1)
    β = np.zeros(n+1)

    v = np.zeros(n)
    v[0] = v_initial

    α[0] = 0
    β[0] = β0

    for i in range(1, n):
        v_t1 = v[i-1]
        v_t2 = 0

        while True:
            Ai = A_func(i, v_t1)
            Bi = B_func(i, v_t1)
            Ci = C_func(i, v_t1)
            Fi = F_func(i, v_t1)

            α[i+1] = -Ci/(α[i]*Ai + Bi)
            β[i+1] = (Fi-Ai*β[i])/(α[i]*Ai + Bi)
            v_t2 = α[i+1]*v_t1 + β[i+1]

            if abs(v_t1 - v_t2) <= ε:
                break

            v_t1 = v_t2

        v[i] = v_t2

        if i % 10 == 0:
            print('-- {} value: {}'.format(i, v[i]))
    return v

def PrintSolution(z, v, title):
    plt.plot(z, v, color='blue', markerfacecolor='blue', linestyle='dashed')
    plt.title(title)
    plt.xlabel('Domain')
    plt.ylabel('Numerical Solution')
    plt.show()

## Generate 'z' values

In [ ]:
import numpy as np

z_left_border = -1.0
z_right_border = 0.0

n=100
h=(z_right_border - z_left_border)/(n-1)

z = np.zeros(n)
for i in range(0, n):
    z[i]=z_left_border + i*h

print(z)

## Set parameters

In [ ]:
τ = 1e-8

δ=1
Bi=-3
Gr=0.03
Pr=0.7

## Calculate 'ω'

In [ ]:
ω_initial = 0

C1_func = lambda i: dP0(z[i], Bi, δ, Gr) - T0(z[i], Bi, δ, Gr)

A_func = lambda i, v_t1: ((δ**2)/(h**2) + (Gr*(δ**2)*v_t1/(2*h)))
B_func = lambda i, v_t1: -2*(δ**2)/(h**2) - 1/τ
C_func = lambda i, v_t1: ((δ**2)/(h**2) - (Gr*(δ**2)*v_t1/(2*h)))
F_func = lambda i, v_t1: C1_func(i) - v_t1/τ

ω = FindSolution(z, ω_initial, A_func, B_func, C_func, F_func, 0)
PrintSolution(z, ω, 'Numerical solution of ω, where δ={}, Bi={}, Gr={}'.format(δ, Bi, Gr))

## Calculate 'u'

In [ ]:
u_initial = 0

C2_func = lambda i: P11(z[i], Bi, δ, Gr)

A_func = lambda i, v_t1: -(1/(h**2) + (Gr*ω[i]/(2*h)))
B_func = lambda i, v_t1: 1/(h**2) + Gr*v_t1 - 1/τ
C_func = lambda i, v_t1: -(1/(h**2) - (Gr*ω[i]/(2*h)))
F_func = lambda i, v_t1: C2_func(i) - v_t1/τ

u = FindSolution(z, u_initial, A_func, B_func, C_func, F_func, 0)
PrintSolution(z, u, 'Numerical solution of u, where δ={}, Bi={}, Gr={}'.format(δ, Bi, Gr))

## Calculate 'T11'

In [ ]:
t11_initial = -1

A_func = lambda i, v_t1: 1/(Pr*(h**2)) + (Gr*ω[i]/(2*h))
B_func = lambda i, v_t1: -(2/(Pr*(h**2)) + 2*Gr*u[i] + 1/τ)
C_func = lambda i, v_t1: 1/(Pr*(h**2)) - (Gr*ω[i]/(2*h))
F_func = lambda i, v_t1: -v_t1/τ

t11 = FindSolution(z, t11_initial, A_func, B_func, C_func, F_func, -1)
PrintSolution(z[2:], t11[2:], 'Numerical solution of t11, where δ={}, Bi={}, Gr={}'.format(δ, Bi, Gr))

## Calculate 'T0'

In [ ]:
t0_initial = 0.5

A_func = lambda i, v_t1: 1/(Pr*(h**2)) + (Gr*ω[i]/(2*h))
B_func = lambda i, v_t1: -(2/(Pr*(h**2)) + 1/τ)
C_func = lambda i, v_t1: 1/(Pr*(h**2)) - (Gr*ω[i]/(2*h))
F_func = lambda i, v_t1: -v_t1/τ - ((δ**2)*t11[i])/Pr

t0 = FindSolution(z, t0_initial, A_func, B_func, C_func, F_func, 0.5)
PrintSolution(z[1:], t0[1:], 'Numerical solution of t0, where δ={}, Bi={}, Gr={}'.format(δ, Bi, Gr))

## Calculate 'P11'

In [ ]:
p11= np.zeros(n)
p11[-1] = 0

for i in range(2, n):
    k1 = t11[-i]
    k2 = t11[-i] # i-th value of 't11' 
    k3 = z[-i] # i-th value of 'z'
    k4 = 0

    p11[-(i+1)] = p11[-i] + (h/6)*(k1 + 2*k2 + 2*k3 + k4)